#### Pseudo Code

- Convert original pricing PSD to dataframe
- Create dataframe or dictionary containing partnumber, SAP description, and new pricing data
- Iterate through and update pricing in dataframe referencing pricing dictionary/dataframe



- Add 2 columns: 1 to contain original pricing, and 1 to calculate percent difference?
- Write new tabs to excel
"""

#### Imports

In [17]:
import pandas as pd
import os
import openpyxl

#### Convert original pricing PSD to dataframe

In [18]:
# Folder where PSDs are located
fileDir = r"C:\Users\104092\OneDrive - Grundfos\Documents\10-19 Projects\15 Patrick Price Update\15.01 Original PSDs"

file_to_change = r"TPE3 Grundfos_GrundfosPumps Constraint Table PSD.xlsx"
filePath = os.path.join(fileDir, file_to_change)

# Create dataframe from PSD
psd_data = pd.read_excel(filePath, sheet_name=1, header=None, na_filter=False)

new_header = psd_data.iloc[1] # grab the 2nd row for the header
psd_data.columns = new_header # set the header row as the new df header

#### Convert Big Book to dataframe or dictionary containing partnumber, SAP description, and new pricing data

In [19]:
newDir = r"C:\Users\104092\OneDrive - Grundfos\Documents\10-19 Projects\15 Patrick Price Update"
newPrices = r"2023_Big_Book_v.1.xlsx"
pathNewPrices = os.path.join(newDir, newPrices)

# Create dataframe of new prices
big_book_cols = ['Part Number', '2023 List Prices']
newPrice_data = pd.read_excel(pathNewPrices, sheet_name=0, usecols=big_book_cols)

# Renaming part number column for use in pandas merge
newPrice_data.rename(columns = {'Part Number':'Big Book Part Number'}, inplace=True)

#### Reorder df column function

In [20]:
def reorder_columns(dataframe, col_name, position):
    """Reorder a dataframe's column.
    Args:
        dataframe (pd.DataFrame): dataframe to use
        col_name (string): column name to move
        position (0-indexed position): where to relocate column to
    Returns:
        pd.DataFrame: re-assigned dataframe
    """
    temp_col = dataframe[col_name]
    dataframe = dataframe.drop(columns=[col_name])
    dataframe.insert(loc=position, column=col_name, value=temp_col)
    return dataframe

#### Iterate through and update pricing in dataframe referencing pricing dictionary/dataframe

In [21]:
# Essentially a VLOOKUP in python
df = psd_data.merge(newPrice_data, how='left', left_on='PartNumber', right_on='Big Book Part Number')

# Find Column Index of "ListPrice" within excel file (This is where we will insert the new prices column)
insert_loc_col = df.columns.get_loc("ListPrice") + 1

# Re ordering columns for QA review. Insert New Columns (PN, List Price) next to "ListPrice" column for comparison
df = reorder_columns(dataframe=df, col_name=big_book_cols[1], position=insert_loc_col)
df = reorder_columns(dataframe=df, col_name='Big Book Part Number', position=insert_loc_col)

#### Need to Move "ListPrice" Attribute to new 2023 List Price Column

In [22]:
# Clear Old Column List Price Attributes
df.at[1, 'ListPrice'] = ""
df.at[2, 'ListPrice'] = ""

# Set New Column List Price Attributes
df.at[1, big_book_cols[1]] = "ListPrice"
df.at[2, big_book_cols[1]] = "double"
df.at[7, big_book_cols[1]] = big_book_cols[1]

#### Create Working Copy of pricing PSD; leaving original untouched

In [163]:
import shutil

filename, file_ext = file_to_change.split(os.extsep)

outputDir = r"C:\Users\104092\OneDrive - Grundfos\Documents\10-19 Projects\15 Patrick Price Update\15.02 Output Files"
outputFilename = filename + " - Ready for Review." + file_ext
outputPath = os.path.join(outputDir, outputFilename)

shutil.copyfile(filePath, outputPath) # Create copy to leave original untouched

'C:\\Users\\104092\\OneDrive - Grundfos\\Documents\\10-19 Projects\\15 Patrick Price Update\\15.02 Output Files\\TPE3 Grundfos_GrundfosPumps Constraint Table PSD - Ready for Review.xlsx'

#### Write to new tab in excel sheet

In [164]:
from openpyxl import load_workbook
wb = load_workbook(outputPath, data_only=True)
ws = wb.worksheets[1]
sheet_name = ws.title

In [165]:
# Write dataframe 
with pd.ExcelWriter(outputPath, mode="a", engine="openpyxl", if_sheet_exists='replace') as writer:
    # df3.to_excel(writer, sheet_name="CP BpaQ", header=None, startrow=writer.sheets["Sheet1"].max_row, index=False)
    df.to_excel(writer, sheet_name=sheet_name, header=None, index=False)